## Interpreter Pattern

- __Type:__ Behavioral
- __Popularity: ★★☆☆☆__
- __Complexity: ★★★★☆__

### Intent:
__Interpreter__ is a behavioral design pattern that defines a grammatical representation for a language and provides an interpreter to process sentences in this language. It allows you to evaluate expressions or statements written in a specific domain-specific language.

### Problem:
You need to process and evaluate expressions in a specialized language, grammar, or notation. Standard parsing and interpretation approaches might be too complex or inefficient for your needs. This often occurs when:
- You need to implement a domain-specific language (DSL) for a particular application
- You need to process structured text input according to specific grammar rules
- You need to evaluate expressions with well-defined rules within your application
- The grammar is simple enough that a full compiler or parser generator would be overkill

### Solution:
The Interpreter pattern solves this by defining a class hierarchy that represents the grammar rules of the language to be interpreted. Each rule in the grammar becomes an expression that can be evaluated. The pattern:

- Represents each grammar rule as a class
- Organizes grammar classes into an Abstract Syntax Tree (AST) structure
- Uses composition to represent complex expressions made of simpler ones
- Implements an interpret method on each expression class that recursively evaluates expressions
- Typically uses a context object to store and track state during interpretation

This approach allows you to parse and evaluate expressions in a structured, object-oriented way that mirrors the grammar's structure.

### Diagram:
```mermaid
classDiagram
    class AbstractExpression {
        +interpret(context)
    }
    class TerminalExpression {
        +interpret(context)
    }
    class NonterminalExpression {
        -expressions: AbstractExpression[]
        +interpret(context)
    }
    class Context {
        -data
        +lookup()
        +assign()
    }
    
    AbstractExpression <|-- TerminalExpression
    AbstractExpression <|-- NonterminalExpression
    NonterminalExpression o-- AbstractExpression
    AbstractExpression ..> Context
```

In [2]:
from abc import ABC, abstractmethod


# Abstract Expression
class Expression(ABC):
    @abstractmethod
    def interpret(self, context) -> bool:
        pass

In [3]:
# Terminal Expressions
class VariableExpression(Expression):
    def __init__(self, variable: str):
        self.variable = variable

    def interpret(self, context) -> bool:
        return context.get(self.variable, False)

In [4]:
# Non-terminal Expressions
class AndExpression(Expression):
    def __init__(self, expr1: Expression, expr2: Expression):
        self.expr1 = expr1
        self.expr2 = expr2

    def interpret(self, context) -> bool:
        return self.expr1.interpret(context) and self.expr2.interpret(context)


class OrExpression(Expression):
    def __init__(self, expr1: Expression, expr2: Expression):
        self.expr1 = expr1
        self.expr2 = expr2

    def interpret(self, context) -> bool:
        return self.expr1.interpret(context) or self.expr2.interpret(context)


class NotExpression(Expression):
    def __init__(self, expr: Expression):
        self.expr = expr

    def interpret(self, context) -> bool:
        return not self.expr.interpret(context)

In [5]:
# Context class to store variable values
class Context(dict):
    def __init__(self, variables=None):
        super().__init__()
        if variables:
            for var, value in variables.items():
                self[var] = value

In [6]:
# Client code
if __name__ == "__main__":
    # Define expressions
    # Rule: (A && B) || (!C)
    a = VariableExpression("A")
    b = VariableExpression("B")
    c = VariableExpression("C")

    not_c = NotExpression(c)
    a_and_b = AndExpression(a, b)
    expression = OrExpression(a_and_b, not_c)

    # Create a context
    context1 = Context({"A": True, "B": True, "C": True})
    context2 = Context({"A": True, "B": False, "C": True})
    context3 = Context({"A": True, "B": False, "C": False})

    # Interpret expressions
    print(f"Rule: (A && B) || (!C)")
    print(f"Context: {context1}")
    print(f"Result: {expression.interpret(context1)}\n")

    print(f"Context: {context2}")
    print(f"Result: {expression.interpret(context2)}\n")

    print(f"Context: {context3}")
    print(f"Result: {expression.interpret(context3)}")

Rule: (A && B) || (!C)
Context: {'A': True, 'B': True, 'C': True}
Result: True

Context: {'A': True, 'B': False, 'C': True}
Result: False

Context: {'A': True, 'B': False, 'C': False}
Result: True


### Example 2: Smart Home Command Interpreter with PyParsing

In [7]:
from pyparsing import Group, OneOrMore, Optional, Suppress, Word, alphanums


class Gate:
    def __init__(self):
        self.is_open = False

    def __str__(self):
        return "open" if self.is_open else "closed"

    def open(self):
        print("opening the gate")
        self.is_open = True

    def close(self):
        print("closing the gate")
        self.is_open = False


class Garage:
    def __init__(self):
        self.is_open = False

    def __str__(self):
        return "open" if self.is_open else "closed"

    def open(self):
        print("opening the garage")
        self.is_open = True

    def close(self):
        print("closing the garage")
        self.is_open = False


class Aircondition:
    def __init__(self):
        self.is_on = False

    def __str__(self):
        return "on" if self.is_on else "off"

    def turn_on(self):
        print("turning on the air condition")
        self.is_on = True

    def turn_off(self):
        print("turning off the air condition")
        self.is_on = False


class Heating:
    def __init__(self):
        self.is_on = False

    def __str__(self):
        return "on" if self.is_on else "off"

    def turn_on(self):
        print("turning on the heating")
        self.is_on = True

    def turn_off(self):
        print("turning off the heating")
        self.is_on = False


class Boiler:
    def __init__(self):
        self.temperature = 83  # in celsius

    def __str__(self):
        return f"boiler temperature: {self.temperature}"

    def increase_temperature(self, amount):
        print(f"increasing the boiler's temperature by {amount} degrees")
        self.temperature += amount

    def decrease_temperature(self, amount):
        print(f"decreasing the boiler's temperature by {amount} degrees")
        self.temperature -= amount


class Fridge:
    def __init__(self):
        self.temperature = 2  # in celsius

    def __str__(self):
        return f"fridge temperature: {self.temperature}"

    def increase_temperature(self, amount):
        print(f"increasing the fridge's temperature by {amount} degrees")
        self.temperature += amount

    def decrease_temperature(self, amount):
        print(f"decreasing the fridge's temperature by {amount} degrees")
        self.temperature -= amount

In [8]:
# Main interpreter function
def main():
    # Define the grammar for our home automation language
    word = Word(alphanums)
    command = Group(OneOrMore(word))
    token = Suppress("->")
    device = Group(OneOrMore(word))
    argument = Group(OneOrMore(word))
    event = command + token + device + Optional(token + argument)

    # Initialize devices
    gate = Gate()
    garage = Garage()
    airco = Aircondition()
    heating = Heating()
    boiler = Boiler()
    fridge = Fridge()

    # Device lookup dictionary
    devices = {
        "gate": gate,
        "garage": garage,
        "air condition": airco,
        "heating": heating,
        "boiler temperature": boiler,
        "fridge temperature": fridge,
    }

    # Test commands
    tests = (
        "open -> gate",
        "close -> garage",
        "turn on -> air condition",
        "turn off -> heating",
        "increase -> boiler temperature -> 5 degrees",
        "decrease -> fridge temperature -> 2 degrees",
    )

    # Define actions available for each device
    open_actions = {
        "gate": gate.open,
        "garage": garage.open,
        "air condition": airco.turn_on,
        "heating": heating.turn_on,
        "boiler temperature": boiler.increase_temperature,
        "fridge temperature": fridge.increase_temperature,
    }
    close_actions = {
        "gate": gate.close,
        "garage": garage.close,
        "air condition": airco.turn_off,
        "heating": heating.turn_off,
        "boiler temperature": boiler.decrease_temperature,
        "fridge temperature": fridge.decrease_temperature,
    }

    # Execute each command
    for t in tests:
        print(f"\nExecuting: '{t}'")
        if len(event.parseString(t)) == 2:
            cmd, dev = event.parseString(t)
            cmd_str, dev_str = " ".join(cmd), " ".join(dev)
            if "open" in cmd_str or "turn on" in cmd_str:
                open_actions[dev_str]()
            elif "close" in cmd_str or "turn off" in cmd_str:
                close_actions[dev_str]()
        elif len(event.parseString(t)) == 3:
            cmd, dev, arg = event.parseString(t)
            cmd_str = " ".join(cmd)
            dev_str = " ".join(dev)
            arg_str = " ".join(arg)
            num_arg = 0
            try:
                # extract the numeric part
                num_arg = int(arg_str.split()[0])
            except ValueError:
                print(f"expected number but got: '{arg_str[0]}'")
            if "increase" in cmd_str and num_arg > 0:
                open_actions[dev_str](num_arg)
            elif "decrease" in cmd_str and num_arg > 0:
                close_actions[dev_str](num_arg)

        # Safe way to get device status
        device_obj = devices[dev_str]
        print(f"Status after command: {dev_str} is {device_obj}")

In [9]:
# Run the interpreter
if __name__ == "__main__":
    main()


Executing: 'open -> gate'
opening the gate
Status after command: gate is open

Executing: 'close -> garage'
closing the garage
Status after command: garage is closed

Executing: 'turn on -> air condition'
turning on the air condition
Status after command: air condition is on

Executing: 'turn off -> heating'
turning off the heating
Status after command: heating is off

Executing: 'increase -> boiler temperature -> 5 degrees'
increasing the boiler's temperature by 5 degrees
Status after command: boiler temperature is boiler temperature: 88

Executing: 'decrease -> fridge temperature -> 2 degrees'
decreasing the fridge's temperature by 2 degrees
Status after command: fridge temperature is fridge temperature: 0


### Real-world analogies:

1. **Music Notation**:
   
   Music notation is a language where symbols represent notes, duration, and style. Musicians interpret these symbols (the "grammar") to produce music. Each symbol is like a terminal expression, while combinations of notes form non-terminal expressions. The musician serves as the interpreter, evaluating each symbol in the context of the piece.

2. **Court Interpreter**:
   
   A court interpreter translates legal proceedings between languages. The interpreter understands both the grammar rules (sentence structures) and the context (legal terminology). As statements are made, the interpreter processes each component according to established rules to produce an equivalent statement in the target language.

### When to use:
- When you need to implement a simple language or notation with a well-defined grammar
- When the grammar can be represented naturally as a tree structure
- When you want to evaluate expressions in a specific domain
- When performance is not a critical concern (interpreter can be slow for complex grammars)
- When the grammar is relatively stable and won't change frequently
- When you need to process structured text input according to specific rules

### Python-specific implementation notes:
- Python's dynamic typing makes it easier to implement interpreters with less boilerplate code
- Libraries like pyparsing, PLY, and ANTLR provide powerful tools for language parsing in Python
- Python's operator overloading allows for creating more intuitive syntax for expressions
- The `ast` module in Python's standard library provides utilities for working with Python's own abstract syntax trees
- Python's `eval()` and `exec()` functions provide built-in interpretation capabilities, though they should be used carefully due to security concerns

### Related patterns:
- **Composite Pattern**: Interpreter often uses Composite to build the syntax tree of expressions
- **Visitor Pattern**: Can be used to add operations to the interpreter without changing expression classes
- **Iterator Pattern**: Often used for traversing the nodes of the abstract syntax tree
- **Flyweight Pattern**: Can be used to share terminal symbols in a grammar to reduce memory usage
- **Factory Method**: Can be used to create different expression objects based on input tokens